<a href="https://colab.research.google.com/github/good5229/DeepLearning_Project/blob/main/DL_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
import pandas as pd
from pandas import read_csv
from matplotlib import pyplot
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
import itertools
import os

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd /content/drive/MyDrive/Cleansed_and_removed/

#경로 확인
!pwd

/content/drive/MyDrive/Cleansed_and_removed
/content/drive/MyDrive/Cleansed_and_removed


# 훈련용 이미지와 테스트용 이미지 분리

In [14]:
df = pd.read_csv('./preprocessed_art.csv')
df = df.drop('Unnamed: 0', axis=1)
print(len(df))
df.head()

22444


,index,title,author,year,standard,genre,technique,source,date,price,...,MSCI_world_MACD,One_year_bond_momentum,One_year_bond_RSI,One_year_bond_MACD,Gold_momentum,Gold_RSI,Gold_MACD,VIX_momentum,VIX_RSI,VIX_MACD
0,71,무제,1,2003,60.5×91cm,0,캔버스에 유채,1,2021-06-03,18000000,...,-13.005561,0.205508,0.097902,-0.040734,-0.022622,0.398363,26.361374,-0.022622,0.398363,26.361374
1,72,무제,1,2003,27×23cm,0,"종이에 수묵, 수채",0,2021-05-24,1100000,...,-15.280810,0.252033,0.589744,-0.053255,-0.049933,0.137980,23.169392,-0.049933,0.137980,23.169392
2,73,청호박과 호박꽃,1,2002,53×72.7cm,0,캔버스에 아크릴릭,0,2021-04-28,25000000,...,-8.150539,-0.025714,0.560976,0.000158,0.056889,0.398558,-21.732597,0.056889,0.398558,-21.732597
3,74,가을연꽃,1,2007,72.5×49.5cm,0,캔버스에 아크릴릭,1,2021-04-27,17500000,...,-6.991066,0.000000,0.322581,0.000705,0.061353,0.410293,-22.180523,0.061353,0.410293,-22.180523
4,75,만년일파,1,2012,91×116.8cm,0,캔버스에 아크릴릭,0,2020-11-25,20000000,...,-25.257254,-0.025806,0.480000,0.006673,0.059156,0.800358,-16.623125,0.059156,0.800358,-16.623125


In [15]:
mask_1 = (df['date'] >= '2016-01-01') & (df['date'] <= '2020-12-31')
df_train = df.loc[mask_1]
df_train = df_train.reset_index()

mask_2 = (df['date'] >= '2021-01-01')
df_test = df.loc[mask_2]
df_test = df_test.reset_index()

print(len(df_train), len(df_test))

18403 2802


In [16]:
mask_false = (df['date'] <= '2015-12-31')
df_false = df.loc[mask_false]
df_false

,index,title,author,year,standard,genre,technique,source,date,price,...,MSCI_world_MACD,One_year_bond_momentum,One_year_bond_RSI,One_year_bond_MACD,Gold_momentum,Gold_RSI,Gold_MACD,VIX_momentum,VIX_RSI,VIX_MACD
40,111,보리밭,1,2007,112.1×162.2cm,0,캔버스에 아크릴릭,0,2015-12-30,30000000,...,28.983204,-0.105691,0.736264,0.023887,0.113604,0.566325,-24.327257,0.113604,0.566325,-24.327257
41,112,삼태성,1,2007,162×130.2cm,0,캔버스에 아크릴릭,1,2015-12-16,60000000,...,27.439848,-0.058380,0.591954,0.027949,0.046775,0.526238,-14.918292,0.046775,0.526238,-14.918292
42,113,썰물,1,1994,72.8×116.8cm,0,캔버스에 유채,1,2015-12-16,23000000,...,27.439848,-0.058380,0.591954,0.027949,0.046775,0.526238,-14.918292,0.046775,0.526238,-14.918292
43,117,텃밭,1,1991,44×74cm,0,캔버스에 유채,0,2015-11-10,4500000,...,7.736406,0.010385,0.045455,-0.005680,-0.016366,0.923566,2.159048,-0.016366,0.923566,2.159048
44,118,용폭,1,2005,227×181.6cm,0,캔버스에 아크릴릭,1,2015-09-15,60000000,...,-8.332418,0.018100,0.750000,-0.005974,0.052744,0.538204,-4.488522,0.052744,0.538204,-4.488522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22439,29013,새벽 눈,158,1998,53×72.7cm,0,캔버스에 유채,0,2015-12-30,9500000,...,28.983204,-0.105691,0.736264,0.023887,0.113604,0.566325,-24.327257,0.113604,0.566325,-24.327257
22440,29014,풍경(선탄장),158,1986,72.7×60.5cm,0,캔버스에 유채,1,2015-12-16,15000000,...,27.439848,-0.058380,0.591954,0.027949,0.046775,0.526238,-14.918292,0.046775,0.526238,-14.918292
22441,29015,풍경,158,1986,72.8×91cm,0,캔버스에 유채,1,2015-12-16,15000000,...,27.439848,-0.058380,0.591954,0.027949,0.046775,0.526238,-14.918292,0.046775,0.526238,-14.918292
22442,29016,고무 씹기,158,2008,72.2×60cm,0,캔버스에 유채,1,2015-06-16,45000000,...,9.338563,-0.034069,0.688000,0.010046,-0.078979,0.523847,15.606383,-0.078979,0.523847,15.606383


In [22]:
false_list = list(df_false['index'].values)
len(false_list)

1239

In [25]:
image_path = './'
file_list = os.listdir(image_path)
print(len(file_list))

22448


22444


In [36]:
def file_split():
    file_idx_list = set()
    for file in file_list:
        if file.endswith('.png'):
            file_idx = file.split('_')[0]
            file_idx_list.add(int(file_idx))


    false_list = list(map(int, false_list))
    type(false_list[0])

    file_source = './'
    file_destination = './unused/'
    counter = 0
    for idx in false_list:
        for file in file_list:
            if str(idx)==file.split('_')[0]:
                counter+=1
                # print(file)
                os.replace(file_source + file, file_destination + file)
    train_list = list(df_train['index'].values)
    train_list = list(map(int, train_list))
    print(len(train_list))

    file_source = './'
    file_destination = './train/'
    counter = 0
    for idx in train_list:
        for file in file_list:
            if str(idx)==file.split('_')[0]:
                counter+=1
                # print(file)
                os.replace(file_source + file, file_destination + file)

    test_list = list(df_test['index'].values)
    test_list = list(map(int, test_list))
    print(len(test_list))

    file_source = './'
    file_destination = './test/'
    counter = 0
    for idx in test_list:
        for file in file_list:
            if str(idx)==file.split('_')[0]:
                counter+=1
                # print(file)
                os.replace(file_source + file, file_destination + file)

int

In [48]:
false_image_path = './unused/'
false_file_list = os.listdir(false_image_path)

train_image_path = './train/'
train_file_list = os.listdir(train_image_path)

test_image_path = './test/'
test_file_list = os.listdir(test_image_path)

1239
18403
2802


In [50]:
test_file_list[:5]

['891_음양8S76_김구림_2008.png',
 '892_무제_김구림_1978.png',
 '896_무제_김구림_1989.png',
 '897_무제_김구림_1981.png',
 '893_무제_김구림_1989.png']